# Microsoft Face Api Integration through Python 

Build facial recognition software into your applications with the Face API from Microsoft Azure. Detect, identify and verify faces with this powerful API. I will try to integrate these APIs through python. For this, I will create a small face dataset of all the characters of our favorite sitcom F.R.I.E.N.D.S. I will use this dataset to identify whether the image provided by me belongs to which character. The same logic can be used for creating projects like Attendance management, visitor management, access control etc with facial recognition.  

Let's start with running the most basic of these APIs: Face - Detect

### Face - Detect

Detects human faces in an image, return face rectangles, and optionally with faceIds, landmarks, and attributes.

In [2]:
# import modules
import requests

In [3]:
# Request headers set Subscription key which provides access to this API. Found in your Cognitive Services accounts.
headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': 'XXXad1b1f6564XXXXed71a61f7XXXb44',
}

body = dict()
body["url"] = "http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659"
body = str(body)

# Request URL 
FaceApiDetect = 'https://westus.api.cognitive.microsoft.com/face/v1.0/detect?returnFaceId=true&returnFaceAttributes=age,gender,headPose,smile,facialHair' 

try:
    # REST Call 
    response = requests.post(FaceApiDetect, data=body, headers=headers) 
    print("RESPONSE:" + str(response.json()))

except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

RESPONSE:[{'faceId': '2bed5571-71e8-4930-9fae-b2ba46db077d', 'faceRectangle': {'top': 335, 'left': 249, 'width': 319, 'height': 319}, 'faceAttributes': {'smile': 1.0, 'headPose': {'pitch': 0.0, 'roll': -3.7, 'yaw': -2.6}, 'gender': 'male', 'age': 43.0, 'facialHair': {'moustache': 0.1, 'beard': 0.1, 'sideburns': 0.1}}}]


A successful call returns an array of face entries ranked by face rectangle size in descending order. An empty response indicates no faces detected. A face entry may contain the following values depending on input parameters: faceId, faceRectangle, faceAttributes, etc. Matthew Perry AKA chandler seems to be smiling in this photo, API has got that right but not so sure about his age.

Alright, now to our original problem statement. The first step is to create a database of face images of all our friend's characters and train a model. Which we will use later to identify our friends characters from any given face image.

### Step 1: Create a person group using Face API:  PersonGroup - Create

Create a new person group with specified personGroupId, name, and user-provided userData. 
A person group is the container of the uploaded person data, including face images and face recognition features. 

In [7]:
personGroupId="friends"

body = dict()
body["name"] = "F.R.I.E.N.D.S"
body["userData"] = "All friends cast"
body = str(body)

#Request URL 
FaceApiCreateLargePersonGroup = 'https://westus.api.cognitive.microsoft.com/face/v1.0/persongroups/'+personGroupId 

try:
    # REST Call 
    response = requests.put(FaceApiCreateLargePersonGroup, data=body, headers=headers) 
    print("RESPONSE:" + str(response.status_code))

except Exception as e:
    print(e)

RESPONSE:200


### Step 2: Create a person using face API: PersonGroup Person - Create

Create a new person in a specified person group. This will return a personId which we will use in the next step to add training face images to this person. we can also save some additional metadata for each person like a name. We will do chandler first because he is, after all, everyone's favorite character.

In [8]:
body = dict()
body["name"] = "Chandler"
body["userData"] = "Friends"
body = str(body)

#Request URL 
FaceApiCreatePerson = 'https://westus.api.cognitive.microsoft.com/face/v1.0/persongroups/'+personGroupId+'/persons' 

try:
    # REST Call 
    response = requests.post(FaceApiCreatePerson, data=body, headers=headers) 
    responseJson = response.json()
    personId = responseJson["personId"]
    print("PERSONID: "+str(personId))
    
except Exception as e:
    print(e)

PERSONID: cd337328-4a7b-4316-aba8-91f7d7886bfb


### Step 3: Add face images to the person using face API: PersonGroup Person - Add Face

Add a face image to a person into a person group for face identification or verification. To deal with the image of multiple faces, input face can be specified as an image with a targetFace rectangle. It returns a persistedFaceId representing the added face. The extracted face feature, instead of the actual image, will be stored on the server until PersonGroup PersonFace - Delete, PersonGroup Person - Delete or PersonGroup - Delete is called.

In [9]:
chandlerImageList = ["http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659",
                     "https://i.pinimg.com/236x/b0/57/ff/b057ff0d16bd5205e4d3142e10f03394--muriel-matthew-perry.jpg",
                     "https://qph.fs.quoracdn.net/main-qimg-74677a162a39c79d6a9aa2b11cc195b1",
                     "https://pbs.twimg.com/profile_images/2991381736/e2160154f215a325b0fc73f866039311_400x400.jpeg",
                     "https://i.pinimg.com/236x/f2/9f/45/f29f45049768ddf5c5d75ff37ffbfb3f--hottest-actors-matthew-perry.jpg"]

#Request URL 
FaceApiCreatePerson = 'https://westus.api.cognitive.microsoft.com/face/v1.0/persongroups/'+personGroupId+'/persons/'+personId+'/persistedFaces' 

for image in chandlerImageList:
    body = dict()
    body["url"] = image
    body = str(body)

    try:
        # REST Call 
        response = requests.post(FaceApiCreatePerson, data=body, headers=headers) 
        responseJson = response.json()
        persistedFaceId = responseJson["persistedFaceId"]
        print("PERSISTED FACE ID: "+str(persistedFaceId))
    
    except Exception as e:
        print(e)

PERSISTED FACE ID: 4093cdd9-2fa1-4d7f-8d7a-09d7488ddc9f
PERSISTED FACE ID: 087e27db-9c06-4f1d-9e10-1099fc5bc091
PERSISTED FACE ID: a4abe7a0-7619-4841-addb-ae7d5421fa57
PERSISTED FACE ID: 57873f5c-1fa6-44db-8f93-b6d32b9ab102
PERSISTED FACE ID: acf36ddb-62ad-4a43-9194-5ca05b0e6228


##### Note: Repeat the step 2 and 3 to add  face images of the remaining characters.

### Step 4:  Train the model using face API: PersonGroup - Train

Submit a person group training task. Training is a crucial step that only a trained person group can be used by Face - Identify. 

The training task is an asynchronous task. Training time depends on the number of person entries, and their faces in a person group. It could be several seconds to minutes. To check training status, please use PersonGroup - Get Training Status.

In [10]:
body = dict()

#Request URL 
FaceApiTrain = 'https://westus.api.cognitive.microsoft.com/face/v1.0/persongroups/'+personGroupId+'/train'

try:
    # REST Call 
    response = requests.post(FaceApiTrain, data=body, headers=headers) 
    print("RESPONSE:" + str(response.status_code))

except Exception as e:
    print(e)

RESPONSE:202


### Step 5:  Final Step Identify face image. This involves in total 3 face Apis.

#### 5.1 First API: Face - Detect

This will return a face Id which is valid for 24 hrs. This face id will be used by the next API to identify the character. Let's try to give new random chandlers face image and see if the model identifies it accurately.

In [11]:
body = dict()
body["url"] = "https://upload.wikimedia.org/wikipedia/en/thumb/6/6c/Matthew_Perry_as_Chandler_Bing.jpg/220px-Matthew_Perry_as_Chandler_Bing.jpg"
body = str(body)

# Request URL 
FaceApiDetect = 'https://westus.api.cognitive.microsoft.com/face/v1.0/detect?returnFaceId=true' 

try:
    # REST Call 
    response = requests.post(FaceApiDetect, data=body, headers=headers) 
    responseJson = response.json()
    faceId = responseJson[0]["faceId"]
    print("FACE ID: "+str(faceId))

except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

FACE ID: 8f1ec32d-60b3-4b86-a4d4-e8933463b734


#### 5.2 Second API: Face - Identify

1-to-many identification to find the closest matches of the specific query person face from a person group. For each face in the faceIds array, Face Identify will compute similarities between the query face and all the faces in the person group (given by personGroupId), and return candidate person(s) for that face ranked by similarity confidence.

This is will return the personId which we will use in the next optional step to find out the metadata stored with that person id like his name.

In [12]:
faceIdsList = [faceId]

body = dict()
body["personGroupId"] = personGroupId
body["faceIds"] = faceIdsList
body["maxNumOfCandidatesReturned"] = 1 
body["confidenceThreshold"] = 0.5
body = str(body)

# Request URL 
FaceApiIdentify = 'https://westus.api.cognitive.microsoft.com/face/v1.0/identify' 

try:
    # REST Call 
    response = requests.post(FaceApiIdentify, data=body, headers=headers) 
    responseJson = response.json()
    personId = responseJson[0]["candidates"][0]["personId"]
    confidence = responseJson[0]["candidates"][0]["confidence"]
    print("PERSON ID: "+str(personId)+ ", CONFIDENCE :"+str(confidence))
        
except Exception as e:
    print("Could not detect")

PERSON ID: cd337328-4a7b-4316-aba8-91f7d7886bfb, CONFIDENCE :0.79974


#### 5.3 Second Api: PersonGroup Person - Get

Retrieve a person's name and userData, and the persisted faceIds representing the registered person face image.

In [13]:
# Request URL 
FaceApiGetPerson = 'https://westus.api.cognitive.microsoft.com/face/v1.0/persongroups/'+personGroupId+'/persons/'+personId

try:
    response = requests.get(FaceApiGetPerson, headers=headers) 
    responseJson = response.json()
    print("This Is "+str(responseJson["name"]))
    
except Exception as e:
    print("Could not detect")

This Is Chandler
